# Project - Draw my story

Create an interactive story with a few prompts from a toddler and create a fun story, with interactive prompts that are stored using tools and with a bonus super fun image at the end!

In [ ]:
# imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

In [65]:
system_message = """
You are a story telling assistant interacting with a toddler focussed on story telling. Decline politely if 
any prompt that is not suitable for small kids. Once you have all the prompts in the story, the story should always 
begin with this text - Once upon a time in a magical world, 
"""

Create a dictionary to store story elements that can be used for building a story in case there is no prompt from toddler, which is quite common.

In [ ]:
# Initialize dictionary to store story elements
story_data = {
    "character": [],
    "colors": [],
    "places": [],
    "feelings": [],
    "Actions": []
}
print("Story data dictionary initialized successfully!")

In [ ]:
# Populate dictionary with toddler-friendly popular values
story_data["character"] = ["Cat", "Dog", "Unicorn", "Elephant"]
story_data["colors"] = ["Orange", "Blue", "Pink", "Green"]
story_data["places"] = ["Castle", "Forest", "Magic Land", "City"]
story_data["feelings"] = ["Sad", "Excited", "Joyful", "Brave"]
story_data["Actions"] = ["Dance", "Run", "Fly", "Eat"]

print(f"Successfully populated dictionary with {len(story_data['character'])} toddler-friendly entries!")
print("\nDictionary contents:")
for key, values in story_data.items():
    print(f"{key}: {values}")

In [ ]:
story_data

Tool: Get and store 5 prompts from the user - 1. character 2. colors 3. places 4. feelings & 5. Actions

In [22]:
# LLM Tool Function Definition
story_function = {
    "type": "function",
    "function": {
        "name": "get_story_elements",
        "description": "Get story elements from a toddler user: character, colors, places, feelings, and actions to create a personalized story",
        "parameters": {
            "type": "object",
            "properties": {
                "character": {
                    "type": "string",
                    "description": "The main character in the story (e.g., princess, dinosaur, unicorn, superhero)"
                },
                "colors": {
                    "type": "string",
                    "description": "Colors to include in the story (e.g., pink, blue, rainbow, green)"
                },
                "places": {
                    "type": "string",
                    "description": "The place or setting where the story takes place (e.g., castle, forest, magic land, city)"
                },
                "feelings": {
                    "type": "string",
                    "description": "The feeling or emotion in the story (e.g., happy, excited, joyful, brave)"
                },
                "Actions": {
                    "type": "string",
                    "description": "The action or activity happening in the story (e.g., dancing, running, flying, saving)"
                }
            },
            "required": ["character", "colors", "places", "feelings", "Actions"],
            "additionalProperties": False
        }
    }
}


In [21]:
tools = [story_function]

# Time to go multi modal OpenAI work flow

In [57]:
# Some imports for handling images
import base64
import json
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont

In [ ]:
def artist(story):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An toddler friendly image portraying this story: {story}",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))



Create an image for every prompt, so that toddler is engaged whenever he/she tells a prompt

In [76]:
def artist_local(story):
    """
    Create an image with the story text using ImageDraw (no LLM required).
    Creates a colorful, toddler-friendly image with the story text. This is
    """
    # Image dimensions
    width, height = 1024, 1024
    
    # Create a colorful background (toddler-friendly pastel colors)
    # Use a gradient-like effect with multiple colors
    img = Image.new('RGB', (width, height), color=(255, 240, 245))  # Light pink background
    draw = ImageDraw.Draw(img)
    
    # Draw a colorful border/background pattern
    colors = [(255, 200, 220), (200, 220, 255), (220, 255, 200), (255, 255, 200)]
    for i, color in enumerate(colors):
        draw.rectangle([i*20, i*20, width-i*20, height-i*20], outline=color, width=10)
    
    # Try to use a nice font, fallback to default if not available
    try:
        # Try to use a larger, more readable font
        font_size = 40
        font = ImageFont.truetype("arial.ttf", font_size)
        title_font = ImageFont.truetype("arial.ttf", 60)
    except:
        try:
            font = ImageFont.truetype("/System/Library/Fonts/Helvetica.ttc", 40)
            title_font = ImageFont.truetype("/System/Library/Fonts/Helvetica.ttc", 60)
        except:
            # Fallback to default font
            font = ImageFont.load_default()
            title_font = ImageFont.load_default()
    
    # Title
    title = "My Story"
    title_bbox = draw.textbbox((0, 0), title, font=title_font)
    title_width = title_bbox[2] - title_bbox[0]
    title_x = (width - title_width) // 2
    draw.text((title_x, 50), title, fill=(100, 50, 150), font=title_font)
    
    # Text wrapping function
    def wrap_text(text, max_width):
        words = text.split()
        lines = []
        current_line = []
        
        for word in words:
            test_line = ' '.join(current_line + [word])
            bbox = draw.textbbox((0, 0), test_line, font=font)
            test_width = bbox[2] - bbox[0]
            
            if test_width <= max_width:
                current_line.append(word)
            else:
                if current_line:
                    lines.append(' '.join(current_line))
                current_line = [word]
        
        if current_line:
            lines.append(' '.join(current_line))
        
        return lines
    
    # Wrap the story text
    margin = 80
    max_text_width = width - 2 * margin
    lines = wrap_text(story, max_text_width)
    
    # Draw the story text line by line
    y_position = 150
    line_height = 60
    
    for line in lines:
        if y_position + line_height > height - 100:  # Stop if we run out of space
            break
        draw.text((margin, y_position), line, fill=(50, 50, 50), font=font)
        y_position += line_height
    
    # Add some decorative elements
    # Draw colorful circles in corners
    circle_colors = [(255, 150, 150), (150, 255, 150), (150, 150, 255), (255, 255, 150)]
    positions = [(100, 100), (width-100, 100), (100, height-100), (width-100, height-100)]
    for pos, color in zip(positions, circle_colors):
        draw.ellipse([pos[0]-30, pos[1]-30, pos[0]+30, pos[1]+30], fill=color)
    
    return img



In [ ]:
artist_local("cat")

In [27]:
def talker(message):
    response = openai.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="nova",    # Also, try replacing onyx with alloy or coral
      input=message
    )
    return response.content

# Final code

In [77]:
def chat(history):
    # Handle None or empty history
    if history is None:
        history = []
    
    # Convert history format
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # Handle tool calls in a loop until LLM is done storing all inputs
    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    # After all tool calls are done, get the final story from LLM
    current_response = response.choices[0].message.content
    
    # Generate voice for every response (if there's content)
    voice = talker(current_response)
    
    # Only generate image if this is the final response (all tool calls complete)
    # finish_reason will be "stop" or something other than "tool_calls" when done
    if "Once upon a time in a magical world" in current_response:
        # Generate image only for the final story response
        image = artist(current_response)
    else:
        # For intermediate responses, return None for image
        image = artist_local(current_response)

    # Return updated history with assistant's response
    return history + [{"role": "assistant", "content": current_response}], voice, image

In [ ]:
# Callbacks (along with the chat() function above)

def put_message_in_chatbot(message, history):
        return "", history + [{"role":"user", "content":message}]

# UI definition
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")

# Hooking up events to callbacks

    message.submit(put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, audio_output, image_output]
    )

ui.launch(inbrowser=True)

In [75]:
def handle_tool_calls(message):
    """
    Handle tool calls from the LLM, extract story elements, and store them in the dictionary if they don't exist.
    Returns a list of tool response messages.
    """
    responses = []
    
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_story_elements":
            # Parse the arguments from JSON string
            arguments = json.loads(tool_call.function.arguments)
            character = arguments.get('character')
            colors = arguments.get('colors')
            places = arguments.get('places')
            feelings = arguments.get('feelings')
            Actions = arguments.get('Actions')
            
            # Store in dictionary if they don't exist
            stored_items = []
            if character and character not in story_data["character"]:
                story_data["character"].append(character)
                stored_items.append(f"character: {character}")
            
            if colors and colors not in story_data["colors"]:
                story_data["colors"].append(colors)
                stored_items.append(f"colors: {colors}")
            
            if places and places not in story_data["places"]:
                story_data["places"].append(places)
                stored_items.append(f"places: {places}")
            
            if feelings and feelings not in story_data["feelings"]:
                story_data["feelings"].append(feelings)
                stored_items.append(f"feelings: {feelings}")
            
            if Actions and Actions not in story_data["Actions"]:
                story_data["Actions"].append(Actions)
                stored_items.append(f"Actions: {Actions}")
            
            if stored_items:
                response_content = f"Successfully stored story elements: {', '.join(stored_items)}. Now please create a fun, engaging story using these elements: character={character}, colors={colors}, places={places}, feelings={feelings}, Actions={Actions}"
            else:
                response_content = f"Story elements already exist: {character}, {colors}, {places}, {feelings}, {Actions}. Now please create a fun, engaging story using these elements."
            
            # Create tool response
            responses.append({
                "role": "tool",
                "content": response_content,
                "tool_call_id": tool_call.id
            })
    
    return responses